In [1]:
!pip install ultralytics

In [2]:
import yaml 
import optuna
from ultralytics import YOLO

In [12]:
dataset_path ='/kaggle/input/baseball-rubber-home-glove-2/baseball_rubber_home_glove'

data = {'train' :  f'{dataset_path}/train/images',
        'val' :  f'{dataset_path}/valid/images',
        'test' :  f'{dataset_path}/test/images',
        'nc': 4,
        'names': ['glove','homeplate','baseball','rubber'],
        'class_weights': [9.83, 3.97, 3.84, 4.29]
        }

In [13]:
with open(f'/kaggle/working/baseball_data.yaml', 'w') as f:
    yaml.dump(data, f)

# read the content in .yaml file
with open(f'/kaggle/working/baseball_data.yaml', 'r') as f:
    hamster_yaml = yaml.safe_load(f)
    display(hamster_yaml)

{'class_weights': [9.83, 3.97, 3.84, 4.29],
 'names': ['glove', 'homeplate', 'baseball', 'rubber'],
 'nc': 4,
 'test': '/kaggle/input/baseball-rubber-home-glove-2/baseball_rubber_home_glove/test/images',
 'train': '/kaggle/input/baseball-rubber-home-glove-2/baseball_rubber_home_glove/train/images',
 'val': '/kaggle/input/baseball-rubber-home-glove-2/baseball_rubber_home_glove/valid/images'}

In [14]:
model = YOLO('/kaggle/input/yolo_glove_tracking/pytorch/default/1/glove_tracking_v4_YOLOv11.pt')
baseline_metrics = model.val(data='/kaggle/working/baseball_data.yaml', split='test')
print("Baseline Model Performance:")
print(f"mAP@0.5:0.95: {baseline_metrics.box.map}")
print(f"mAP@0.5: {baseline_metrics.box.map50}")

Ultralytics 8.3.165 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11x summary (fused): 190 layers, 56,832,799 parameters, 0 gradients, 194.4 GFLOPs
val: Fast image access ✅ (ping: 0.1±0.2 ms, read: 543.2±100.9 MB/s, size: 327.4 KB)


val: Scanning /kaggle/input/baseball-rubber-home-glove-2/baseball_rubber_home_glove/test/labels... 87 images, 0 backgrounds, 0 corrupt: 100%|██████████| 87/87 [00:00<00:00, 1099.28it/s]

WARNING ⚠️ val: Cache directory /kaggle/input/baseball-rubber-home-glove-2/baseball_rubber_home_glove/test is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.63it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         87        339      0.817      0.858      0.883      0.398
                 glove         87         87      0.951      0.966      0.978      0.524
             homeplate         87         87      0.635      0.878      0.819      0.375
              baseball         87         87      0.939      0.851      0.929      0.393
                rubber         78         78      0.742      0.737      0.803      0.298
Speed: 3.2ms preprocess, 28.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val6
Baseline Model Performance:
mAP@0.5:0.95: 0.39766293210726517
mAP@0.5: 0.8825436355208935


In [6]:
from ultralytics.data.augment import Albumentations
from ultralytics.utils import LOGGER, colorstr
# import cv2

# def __init__(self, p=1.0):
#         """Initialize the transform object for YOLO bbox formatted params."""
#         self.p = p
#         self.transform = None
#         self.contains_spatial = True  # <-- Add this line
#         prefix = colorstr("albumentations: ")
#         try:
#             import albumentations as A         

#             # Insert required transformation here
#             T = [
#                 # A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
#                 A.RandomRain(p=0.05),
#                 A.RandomSunFlare(p=0.05),
#                 A.RandomShadow(p=0.05),
#                 A.OneOf([A.MotionBlur(blur_limit=5, p=0.3),
#                          A.GaussianBlur(blur_limit=3, p=0.3),
#                          A.MedianBlur(blur_limit=3, p=0.2)
#                         ], p=0.4),
#                 A.CoarseDropout(num_holes_range=[1, 1],hole_height_range=[0.02, 0.08],hole_width_range=[0.02, 0.08],
#                                 fill=50),
#                 A.CLAHE(p=0.02),
#             ]
#             self.transform = A.Compose(T, bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))

#             LOGGER.info(prefix + ", ".join(f"{x}".replace("always_apply=False, ", "") for x in T if x.p))
#         except ImportError:  # package not installed, skip
#             pass
#         except Exception as e:
#             LOGGER.info(f"{prefix}{e}")

# Albumentations.__init__ = __init__

In [ ]:
def create_glove_focused_config():
    """Create training configuration optimized for glove detection"""
    return {
        'epochs': 20,
        
        'imgsz': 800,
        'batch': 16,
        
        'hsv_h': 0.02,      # Slightly more hue variation
        'hsv_s': 0.8,       # Higher saturation variation
        'hsv_v': 0.5,       # More brightness variation
        
        'degrees': 15.0,     # More rotation for glove orientations
        'translate': 0.15,   # Increased translation
        'scale': 0.7,        # More scale variation
        'shear': 5.0,        # Add shear for glove deformation
        'perspective': 0.0005, # Slight perspective changes
        
        'fliplr': 0.3,       # Increased horizontal flip
        'flipud': 0.1,       # Add vertical flip for catching positions
        
        'mosaic': 0.6,       # Increased mosaic for context
        
        'patience': 15,      # More patience for convergence
        'cos_lr': True,      # Cosine learning rate scheduling
        'close_mosaic': 10,  # Close mosaic later

        'freeze': 5
    }


In [16]:
config = create_glove_focused_config()

In [17]:
model = YOLO("/kaggle/input/yolo_glove_tracking/pytorch/default/1/glove_tracking_v4_YOLOv11.pt").to('cuda')
print('Pretrained model loaded')

results = model.train(
        data='/kaggle/working/baseball_data.yaml',
        name='glove_optimized_v2',
        optimizer='auto',
        project='baseball_glove_detection_new_params_weighted',
        **config
    )

Pretrained model loaded
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/kaggle/working/baseball_data.yaml, degrees=15.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.3, flipud=0.1, format=torchscript, fraction=1.0, freeze=5, half=False, hsv_h=0.02, hsv_s=0.8, hsv_v=0.5, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/input/yolo_glove_tracking/pytorch/default/1/glove_tracking_v4_YOLOv11.pt, momentum=0.937, mosaic=0.6, multi_scale=False, name=glove_optimized_v2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0005, plots

train: Scanning /kaggle/input/baseball-rubber-home-glove-2/baseball_rubber_home_glove/train/labels... 3905 images, 20 backgrounds, 0 corrupt: 100%|██████████| 3925/3925 [00:03<00:00, 1140.00it/s]


WARNING ⚠️ train: Cache directory /kaggle/input/baseball-rubber-home-glove-2/baseball_rubber_home_glove/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 201.1±106.7 MB/s, size: 337.6 KB)


val: Scanning /kaggle/input/baseball-rubber-home-glove-2/baseball_rubber_home_glove/valid/labels... 97 images, 1 backgrounds, 0 corrupt: 100%|██████████| 98/98 [00:00<00:00, 549.38it/s]

WARNING ⚠️ val: Cache directory /kaggle/input/baseball-rubber-home-glove-2/baseball_rubber_home_glove/valid is not writeable, cache not saved.


...

      20/20      12.8G      1.513     0.8043      1.065         61        800:  41%|████▏     | 102/246 [02:50<04:01,  1.68s/it]Corrupt JPEG data: 963 extraneous bytes before marker 0xd9
      20/20      12.8G      1.505     0.8034      1.055         20        800: 100%|██████████| 246/246 [06:50<00:00,  1.67s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.35it/s]

                   all         98        371      0.842       0.86      0.889      0.402



20 epochs completed in 2.316 hours.
Optimizer stripped from baseball_glove_detection_new_params_weighted/glove_optimized_v2/weights/last.pt, 114.4MB
Optimizer stripped from baseball_glove_detection_new_params_weighted/glove_optimized_v2/weights/best.pt, 114.4MB

Validating baseball_glove_detection_new_params_weighted/glove_optimized_v2/weights/best.pt...
Ultralytics 8.3.165 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11x summary (fused): 190 layers, 56,831,644 parameters, 0 gradients, 194.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.14it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         98        371      0.858      0.854      0.894      0.403
                 glove         95         95      0.919      0.926      0.933      0.487
             homeplate         91         91      0.845      0.956      0.926       0.42
              baseball         97         97      0.894      0.876      0.905      0.405
                rubber         88         88      0.776      0.659      0.811      0.301
Speed: 0.2ms preprocess, 27.9ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to baseball_glove_detection_new_params_weighted/glove_optimized_v2


In [18]:
fine_tuned_model = YOLO("/kaggle/working/baseball_glove_detection_new_params/glove_optimized_v25/weights/best.pt")
fine_tuned_metrics = fine_tuned_model.val(data='/kaggle/working/baseball_data.yaml', split='test')
print("Fine tuned Model Performance:")
print(f"mAP@0.5:0.95: {fine_tuned_metrics.box.map}")
print(f"mAP@0.5: {fine_tuned_metrics.box.map50}")

Ultralytics 8.3.165 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11x summary (fused): 190 layers, 56,831,644 parameters, 0 gradients, 194.4 GFLOPs
val: Fast image access ✅ (ping: 0.2±0.5 ms, read: 430.1±179.4 MB/s, size: 354.8 KB)


val: Scanning /kaggle/input/baseball-rubber-home-glove-2/baseball_rubber_home_glove/test/labels... 87 images, 0 backgrounds, 0 corrupt: 100%|██████████| 87/87 [00:00<00:00, 872.47it/s]

WARNING ⚠️ val: Cache directory /kaggle/input/baseball-rubber-home-glove-2/baseball_rubber_home_glove/test is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:03<00:00,  1.65it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all         87        339      0.861      0.861      0.893        0.4
                 glove         87         87      0.955      0.943      0.983      0.519
             homeplate         87         87      0.736      0.899       0.84      0.373
              baseball         87         87      0.975      0.887      0.947      0.401
                rubber         78         78      0.777      0.715      0.802      0.306
Speed: 3.3ms preprocess, 28.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val7
Fine tuned Model Performance:
mAP@0.5:0.95: 0.3999816756693894
mAP@0.5: 0.8932046157865192


In [11]:
!zip -r /kaggle/working/baseball_glove_detection_new_params/glove_optimized_v25.zip /kaggle/working/baseball_glove_detection_new_params/glove_optimized_v25

  adding: kaggle/working/baseball_glove_detection_new_params/glove_optimized_v25/ (stored 0%)
  adding: kaggle/working/baseball_glove_detection_new_params/glove_optimized_v25/val_batch0_pred.jpg (deflated 6%)
  adding: kaggle/working/baseball_glove_detection_new_params/glove_optimized_v25/confusion_matrix_normalized.png (deflated 24%)
  adding: kaggle/working/baseball_glove_detection_new_params/glove_optimized_v25/BoxPR_curve.png (deflated 13%)
  adding: kaggle/working/baseball_glove_detection_new_params/glove_optimized_v25/weights/ (stored 0%)
  adding: kaggle/working/baseball_glove_detection_new_params/glove_optimized_v25/weights/best.pt (deflated 8%)
  adding: kaggle/working/baseball_glove_detection_new_params/glove_optimized_v25/weights/last.pt (deflated 8%)
  adding: kaggle/working/baseball_glove_detection_new_params/glove_optimized_v25/val_batch1_labels.jpg (deflated 6%)
  adding: kaggle/working/baseball_glove_detection_new_params/glove_optimized_v25/confusion_matrix.png (deflate